In [7]:
# Step 1: Load the necessary libraries
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import mixed_precision

In [8]:
# Step 2: Check for GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Enable mixed precision training (if supported by your GPU)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


Num GPUs Available:  0


In [9]:
# Step 3: Load the dataset (assuming it's saved as a JSON file)
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)


In [10]:
# Step 4: Keep both 'title' and 'short_description' columns
df = df[['headline', 'short_description', 'category']]

In [11]:
top_10_categories = df['category'].value_counts().head(10).index
df = df[df['category'].isin(top_10_categories)]

In [12]:
# Step 6: Filter the dataset to include only the top 10 frequent categories
df_filtered = df[df['category'].isin(top_10_categories)]

In [13]:
# Step 7: Combine 'title' and 'short_description' into one text column
df_filtered['text'] = df_filtered['headline'] + " " + df_filtered['short_description']


In [14]:
# Step 8: Initialize the Sentence-Transformer model for sentence embeddings and move to GPU
model = SentenceTransformer('all-MiniLM-L6-v2')  # Use GPU if available


C:\Users\71518\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
# Step 9: Generate sentence embeddings for each row in the 'text' column
X = model.encode(df_filtered['text'].tolist(), convert_to_tensor=False)  # List of sentence embeddings


In [ ]:
# Step 10: Encode the target labels ('category') using LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df_filtered['category'])
y = to_categorical(y)  # Convert labels to one-hot encoded format

In [ ]:
# Step 11: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Step 12: Build the Keras neural network model
model_nn = Sequential()

In [ ]:
# Add input layer (Dense layer with 512 units)
model_nn.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))


In [ ]:
# Add dropout layer for regularization
model_nn.add(Dropout(0.5))

In [ ]:
# Add hidden layer
model_nn.add(Dense(256, activation='relu'))


In [ ]:
# Add dropout layer for regularization
model_nn.add(Dropout(0.5))

In [ ]:
# Add output layer with softmax activation (for multi-class classification)
model_nn.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
# Step 13: Compile the model
model_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Step 14: Train the model
history = model_nn.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))


In [ ]:
# Step 15: Evaluate the model
loss, accuracy = model_nn.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

### Test Loss: 0.5399715900421143
### Test Accuracy: 0.8231829404830933